In [ ]:
import pandas as pd
import csv
import string
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine

# Build Supabase engine
DATABASE_DSN = (
    "postgresql://postgres.avcznjglmqhmzqtsrlfg:Czheyuan0227@"
    "aws-0-us-east-2.pooler.supabase.com:6543/postgres?sslmode=require"
)
engine = create_engine(DATABASE_DSN, pool_pre_ping=True)

#SO 
#Backordered是指目前SO尚未出貨的數量，如果用QTY的話會包含已經partial出貨的
# SO = pd.read_csv("Data/open sales orders.csv", encoding="utf-8", encoding_errors="ignore")
SO = pd.read_sql_table("open_sales_orders", con=engine, schema="public") # Pull from Supabase
SO = SO.drop(columns=["Qty"], axis =1)
SO.rename(columns={"Date":"Order Date","Num":"QB Num","Backordered":"Qty(-)"},inplace=True)
SO = SO.drop(SO.columns[[0]], axis =1)
SO = SO.drop(columns=['Type','Due Date','Terms','Amount','Deliv Date','Open Balance',"Invoiced","Rep"], axis =1)
SO = SO.dropna(axis=0, how='all',subset=None, inplace=False)
SO = SO.dropna(thresh=6)
SO['Item']= SO['Item'].str.split(':',expand=True)[1]
SO['Item']= SO['Item'].str.replace("*","")
SO['Qty(+)']="0"
SO['Remark']=""
SO['Order Date']= pd.to_datetime(SO['Order Date'])
SO['Order Date'] = SO['Order Date'].dt.strftime('%Y/%m/%d')
SO['Ship Date']= pd.to_datetime(SO['Ship Date'])
SO['Ship Date'] = SO['Ship Date'].dt.strftime('%Y/%m/%d')
columns = ['Order Date','Ship Date', 'QB Num',"P. O. #","Name",'Qty(+)','Qty(-)', 'Item','Inventory Site','Remark']

#"POD"
pod = pd.read_csv("open purchase orders.csv", encoding="ISO-8859-1")
pod = pod.drop(columns=['Name','Amount','Open Balance',"Rcv'd","Qty"], axis =1)
pod.rename(columns={"Date":"Order Date","Num":"QB Num","Source Name":"Name","Backordered":"Qty(+)"},inplace=True)
pod = pod.drop(pod.columns[[0]], axis =1)
pod = pod.dropna(axis=0, how='all',subset=None, inplace=False)
pod = pod.dropna(thresh=5)
pod['Memo'] = pod['Memo'].str.split(' ',expand=True)[0]
pod['QB Num'] = pod['QB Num'].str.split('(',expand=True)[0]
# print(pod['Memo'].str.split('*',expand=True)[0])
pod['Memo'] = pod['Memo'].str.replace("*","")
pod.rename(columns={"Memo":"Item"},inplace=True)
pod['Order Date']= pd.to_datetime(pod['Order Date'])
pod['Deliv Date']= pd.to_datetime(pod['Deliv Date'])
pod['Order Date'] = pod['Order Date'].dt.strftime('%Y/%m/%d')
pod['Deliv Date'] = pod['Deliv Date'].dt.strftime('%Y/%m/%d')
pod.to_csv('open purchase2.csv',index=False)


#"NAV"
NAV = pd.read_csv("Sales Date return platform.csv", usecols=['Document No.', "Customer PO No.", "Customer Ordering Model",
                                                             "OP Estimated Shipping Date", "Quantity", "No.",
                                                             "Customer Ordering Desc."], encoding='utf-8')
NAV.rename(columns={"Customer PO No.": "QB Num", "Customer Ordering Model": "Item", 'Document No.': "Remark",
                    "OP Estimated Shipping Date": "Ship Date", "Quantity": "Qty(+)"}, inplace=True)
NAV = NAV[NAV['Item'] != 'Engineer Service- COS']
NAV = NAV[NAV['Item'] != 'CUSTOMER SERVICES']
NAV = NAV[NAV['Item'] != 'FORWARDING CHARGE, EXCLUDING IMPORT DUTY.']
# print(NAV['QB Num'])
NAV['QB Num'] = NAV['QB Num'].str.split('(').str[0]

NAV.to_csv('NAV1.csv', index=False)

 
#Inventory
INV = pd.read_sql_table("inventory_status", con=engine, schema="public")

In [7]:
# 讀取 NAV1 並篩選符合條件的數據
s50 = []

for _, row in NAV.iterrows():
    if row['No.'].startswith("S"):  # 檢查 Item 是否以 "S" 開頭
        s50.append(row)

result_lists = []
for original_list in s50:
    # 分割字串
    product_str = original_list[-1]
    product_str = product_str.replace('\u00A0', ' ').replace('\u3000', ' ')
    product_info = product_str.split(', including ')
    #product_info = original_list[-1].split(', including ')
    # print(product_info)
    # product_info[0] = product_info[0].split(',')[0]  # 產品代碼
    components = product_info[1].split(', ') if len(product_info) > 1 else []
    # print(components)
    
    ## Append component to result_lists
    # 建立各組件的新 list
    for component in components:
        new_list = original_list.copy()
        new_list['Customer Ordering Desc.'] = component.strip()
        result_lists.append(new_list)
   
    ## Append base to result_lists
    # 加入產品代碼(Change Desc. to product_info[0])
    new_list_with_product_code = original_list.copy()
    new_list_with_product_code['Customer Ordering Desc.'] = product_info[0]
    result_lists.append(new_list_with_product_code)
    

for i in range(0,len(result_lists)):
    result_lists[i]['Item'] = result_lists[i]['Customer Ordering Desc.']


# 調整數據格式
transformed_lists = []
for result_list in result_lists:
    transformed_list = result_list.copy()
    transformed_list['Item'] = transformed_list['Item'].replace(" ", "")
    
    if len(transformed_list['Item']) > 1 and transformed_list['Item'][1] == 'x' and transformed_list['Item'][0].isdigit():
        quantity = int(transformed_list['Item'].split('x')[0])
        name = transformed_list['Item'].split('x')[-1]
        transformed_list['Item'] = name
        transformed_list['Qty(+)'] = str(quantity * float(transformed_list[4]))  # 更新數量

    transformed_lists.append(transformed_list)

print(pd.DataFrame(transformed_lists))

print(len(transformed_lists))


         Remark      QB Num               No.  \
3    SO25050035  POD-250648  S51-SL1708FF-003   
3    SO25050035  POD-250648  S51-SL1708FF-003   
3    SO25050035  POD-250648  S51-SL1708FF-003   
10   SO25050036  POD-250649  S51-SL1708FF-003   
10   SO25050036  POD-250649  S51-SL1708FF-003   
..          ...         ...               ...   
436  SO25100096  POD-251415  S50-EC500630-PUB   
437  SO25100096  POD-251415  S60-XC803400-027   
437  SO25100096  POD-251415  S60-XC803400-027   
437  SO25100096  POD-251415  S60-XC803400-027   
437  SO25100096  POD-251415  S60-XC803400-027   

                                               Item Qty(+)   Ship Date  \
3                                         i7-9700TE   10.0  2025/11/19   
3    DDR4-16GB-32-SMandM.280-SSD-1TB-PCIe44-TLC5-PN   10.0  2025/11/19   
3                                     SEMIL-1708-FF   10.0  2025/11/19   
10                                        i7-9700TE   10.0  2025/12/17   
10   DDR4-16GB-32-SMandM.280-SSD-1TB-PCIe

C:\Users\Admin\AppData\Local\Temp\ipykernel_36396\1610363184.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  product_str = original_list[-1]
C:\Users\Admin\AppData\Local\Temp\ipykernel_36396\1610363184.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  transformed_list['Qty(+)'] = str(quantity * float(transformed_list[4]))  # 更新數量


### ChatGPT 改良的版本如下

In [6]:
import re
import pandas as pd

# Keep only rows where No. starts with 'S'
df = NAV.loc[NAV['No.'].astype(str).str.startswith('S', na=False)].copy()
df = df.fillna('')

result = []

for rec in df.to_dict('records'):
    # Parse "Customer Ordering Desc."
    desc = (rec['Customer Ordering Desc.']
            .replace('\u00A0', ' ')
            .replace('\u3000', ' ')
            .strip())

    # base + components
    parts = re.split(r',\s*including\s*', desc, maxsplit=1, flags=re.I)
    base = parts[0].split(',', 1)[0].strip()
    comps = [c.strip() for c in parts[1].split(',')] if len(parts) > 1 else []

    # Build rows: components + base (dedup preserves order)
    targets = list(dict.fromkeys([*comps, base]))

    for t in targets:
        new = rec.copy()
        new['Customer Ordering Desc.'] = t
        new['Item'] = t  # mirror

        # Expand patterns like "2xSSD"
        compact = t.replace(' ', '')
        m = re.match(r'^(\d+)x(.+)$', compact)
        if m:
            mult = int(m.group(1))
            new['Item'] = m.group(2)
            try:
                new['Qty(+)'] = float(new['Qty(+)']) * mult
            except Exception:
                new['Qty(+)'] = mult

        result.append(new)

transformed_lists = pd.DataFrame(result, columns=['Remark','QB Num','No.','Item','Qty(+)','Ship Date','Customer Ordering Desc.'])
print(transformed_lists.to_string(index=False))
print(len(transformed_lists))


    Remark     QB Num              No.                                                 Item  Qty(+)  Ship Date                              Customer Ordering Desc.
SO25050035 POD-250648 S51-SL1708FF-003                                            i7-9700TE    10.0 2025/11/19                                            i7-9700TE
SO25050035 POD-250648 S51-SL1708FF-003     DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN    10.0 2025/11/19     DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN
SO25050035 POD-250648 S51-SL1708FF-003                                        SEMIL-1708-FF    10.0 2025/11/19                                        SEMIL-1708-FF
SO25050036 POD-250649 S51-SL1708FF-003                                            i7-9700TE    10.0 2025/12/17                                            i7-9700TE
SO25050036 POD-250649 S51-SL1708FF-003     DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN    10.0 2025/12/17     DDR4-16GB-32-SM and M.280-SSD-1TB-PCIe44-TLC5-PN
SO25050036 POD-2

In [72]:
import os, csv

file = 'NAV1.csv'
rows_to_add = transformed_lists  # your list of rows

with open(file, 'a+', encoding='utf-8', newline='') as f:
    f.seek(0)                                  # move to start to read
    before = sum(1 for _ in csv.reader(f))     # count existing rows (incl. header if any)

    # go back to end to append
    f.seek(0, os.SEEK_END)
    w = csv.writer(f)
    w.writerows(rows_to_add)

    after = before + len(rows_to_add)          # we know how many we appended
    print(f"Rows before append: {before}")
    print(f"Appended: {len(rows_to_add)}")
    print(f"Rows after append: {after}")


Rows before append: 224
Appended: 184
Rows after append: 408


In [73]:
# NAV 加上倉別和日期
mappings = {
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-512GB-PCIe34-TLC5WT-T': 'M.242-SSD-512GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-128GB-PCIe34-TLC5WT-T': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
    'GC-J-A64GB-O-Industrial-Nvidia': 'GC-Jetson-AGX64GB-Orin-Industrial-Nvidia-JetPack-6.0',
    'GC-Jetson-AGX64GB-Orin-Nvidia': 'GC-Jetson-AGX64GB-Orin-Nvidia-JetPack-6.0',
    'AccsyBx-Cardholder-10108GC-5080': 'AccsyBx-Cardholder-10108GC-5080_70_70Ti',
    'Cblkit-FP-NRU-230V-AWP_NRU-240S': 'Cblkit-FP-NRU-230V-AWP_NRU-240S-AWP',
    'E-mPCIe-GPS-M800_Mod_40CM': 'Extnd-mPCIeHS_GPS-M800_Mod_Cbl-40CM_kits',
    'Cbl-M12A5F-OT2-B-Red-Fuse-100CM': 'Cbl-M12A5F-OT2-Black-Red-Fuse-100CM',
    'AccsyBx-Cardholder-9160GC-2000E': 'AccsyBx-Cardholder-9160GC-2000EAda',
    'M.280-SSD-4TB-PCIe4-TLCWT5NH-IK': 'M.280-SSD-4TB-PCIe4-TLCWT5-NH-IK',
    'M.242-SSD-128GB-PCIe34-TLC5WT-T': 'M.242-SSD-128GB-PCIe34-TLC5WT-TD',
    'M.242-SSD-256GB-PCIe34-TLC5WT-T': 'M.242-SSD-256GB-PCIe34-TLC5WT-TD',
    'M.280-SSD-256GB-PCIe44-TLC5WT-T': 'M.280-SSD-256GB-PCIe44-TLC5WT-TD',
    'M.280-SSD-512GB-PCIe44-TLC5WT-T': 'M.280-SSD-512GB-PCIe44-TLC5WT-TD',
    'E-mPCIe-BTWifi-WT-6218_Mod_40CM': 'Extnd-mPCIeHS-BTWifi-WT-6218_Mod_Cbl-40CM_kits',
    'GC-Jetson-NX16G-Orin-Nvidia': 'GC-Jetson-NX16G-Orin-Nvidia-JetPack6.0',
    'FPnl-3Ant-NRU-170-PPC series': 'FPnl-3Ant-NRU-170-PPCseries',
}


NAV = pd.read_csv("NAV1.csv", usecols=['Remark', 'QB Num', 'Item', 'Qty(+)', 'Ship Date'], encoding='utf-8')
NAV['Item'] = NAV['Item'].replace(mappings)
NAV.to_csv('NAV1.csv', index=False)

# 讀取 open purchase2.csv 並處理數據
a = pd.read_csv('open purchase2.csv', usecols=['QB Num', "Order Date", "Inventory Site", "P. O. #", "Name", "Item"])
a.drop_duplicates(inplace=True)
a['Qty(-)'] = "0"

fil = set(a['Item'])
NAV = NAV[NAV['Item'].isin(fil)]
a = a.drop(columns=["Item"])
a.drop_duplicates(inplace=True)

# 合併 NAV 和 open purchase2.csv
Final = pd.merge(left=NAV, right=a, on=["QB Num"], how="left")
columns = ['Order Date', 'Ship Date', 'QB Num', "P. O. #", "Name", 'Qty(-)', 'Qty(+)', 'Item', 'Inventory Site', 'Remark']
Final.to_csv('Final.csv', index=False, columns=columns)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from datetime import date

def build_timephased_qty(
    start_date=None,
    end_date=None,
    freq="D",
    initial_onhand_df=None,   # pass INV here
    site_filter="WH01S-NTA",  # focus on this site
):
    df = SONAV

    # ---- Filter Site ----
    df["Inventory Site"] = df["Inventory Site"].astype(str).str.strip()
    if site_filter is not None:
        df = df[df["Inventory Site"] == site_filter].copy()

    # clean numeric & date
    df["Qty(+)"] = pd.to_numeric(df["Qty(+)"], errors="coerce").fillna(0)
    df["Qty(-)"] = pd.to_numeric(df["Qty(-)"], errors="coerce").fillna(0)
    df["Ship Date"] = pd.to_datetime(df["Ship Date"], errors="coerce")
    df = df.dropna(subset=["Ship Date"])
    df["Item"] = df["Item"].astype(str).str.strip()

    # net movement per date
    df["qty_change"] = df["Qty(+)"] - df["Qty(-)"]
    events = (df.groupby(["Inventory Site","Item","Ship Date"], as_index=False)["qty_change"]
                .sum()
                .sort_values(["Inventory Site","Item","Ship Date"]))

    # horizon
    if start_date is None:
        start = pd.Timestamp("2025-01-01")   # or use today().date()
    else:
        start = pd.to_datetime(start_date).normalize()

    if end_date is None:
        end = start + timedelta(days=30)
    else:
        end = pd.to_datetime(end_date).normalize()

    date_index = pd.date_range(start, end, freq=freq)

    # all (site,item) pairs from events
    pairs = events[["Inventory Site","Item"]].drop_duplicates()

    # build continuous timeline
    all_rows = []
    for (site, item), grp in events.groupby(["Inventory Site","Item"]):
        s = grp.set_index("Ship Date")["qty_change"].reindex(date_index, fill_value=0.0)
        out = pd.DataFrame({
            "Ship Date": date_index,
            "Inventory Site": site,
            "Item": item,
            "Net Movement": s.values
        })
        all_rows.append(out)

    timeline = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame(
        columns=["Ship Date","Inventory Site","Item","Net Movement"]
    )

    # ----- add initial on-hand (INV) -----
    # Case: INV has NO site -> broadcast to the chosen site
    if initial_onhand_df is not None and not initial_onhand_df.empty:
        onhand = initial_onhand_df.copy()
        # normalize columns from INV
        if "Part_Number" in onhand.columns:
            onhand = onhand.rename(columns={"Part_Number":"Item"})
        onhand["Item"] = onhand["Item"].astype(str).str.strip()
        onhand["On Hand"] = pd.to_numeric(onhand["On Hand"], errors="coerce").fillna(0)

        if "Inventory Site" not in onhand.columns:
            onhand["Inventory Site"] = site_filter  # broadcast INV to this site

        onhand = onhand[["Inventory Site","Item","On Hand"]]
        # limit to items present in timeline to avoid bloat
        if not timeline.empty:
            onhand = onhand.merge(pairs, on=["Inventory Site","Item"], how="inner")

        timeline = timeline.merge(onhand, on=["Inventory Site","Item"], how="left")
        timeline["On Hand"] = timeline["On Hand"].fillna(0.0)
    else:
        timeline["On Hand"] = 0.0

    # running balance
    if not timeline.empty:
        timeline.sort_values(["Inventory Site","Item","Ship Date"], inplace=True)
        cum_net = timeline.groupby(["Inventory Site","Item"])["Net Movement"].cumsum()
        init_onhand = timeline.groupby(["Inventory Site","Item"])["On Hand"].transform("first")
        timeline["Projected Qty"] = init_onhand + cum_net
    else:
        timeline["Projected Qty"] = pd.Series(dtype=float)

    return events, timeline

In [ ]:
# Prepare INV (no site column in your screenshot)
initial_onhand_df = INV[["Part_Number","On Hand"]].copy()

events, timeline = build_timephased_qty(
    start_date=date.today(),
    end_date=None,
    freq="D",
    initial_onhand_df=initial_onhand_df,
    site_filter="WH01S-NTA",
)

part_view = timeline[
    (timeline["Item"] == "TB-10") & (timeline["Inventory Site"] == "WH01S-NTA")
]
print(part_view[["Ship Date","On Hand","Net Movement","Projected Qty"]].head(25))